In [ ]:
import librosa
import numpy as np
import sys
import numpy as np
import scipy.linalg
import scipy.stats

In [ ]:
def chromaextract(path):
  y, sr = librosa.load(path, res_type='kaiser_fast')
  y_harmonic = librosa.effects.hpss(y)[0]
  chroma = librosa.feature.chroma_cqt(y=y_harmonic, sr=sr)
  return chroma

In [ ]:
def pitchdistextract(chroma):
  dist = np.zeros(12)
  notes = chroma.argmax(axis=0)
  for i in range(len(notes)):
    for k in range(12):
      if notes[i] == k:
        dist[k] += 1
  dist = dist.astype(int)
  return dist 

In [ ]:
major = np.asarray([6.35, 2.23, 3.48, 2.33, 4.38, 4.09, 2.52, 5.19, 2.39, 3.66, 2.29, 2.88])
minor = np.asarray([6.33, 2.68, 3.52, 5.38, 2.60, 3.53, 2.54, 4.75, 3.98, 2.69, 3.34, 3.17])

In [ ]:
#revised
major = np.asarray([5.0, 2.0, 3.5, 2.0, 4.5, 4.0, 2.0, 4.5, 2.0, 3.5, 1.5, 4.0])
minor = np.asarray([5.0, 2.0, 3.5, 4.5, 2.0, 4.0, 2.0, 4.5, 3.5, 2.0, 1.5, 4.0])

In [ ]:
def step_dist(x):
  y = x[1:]
  y = np.append(y, x[0])
  return y

In [ ]:
def notedistinit(dist):
  nodedist = np.zeros((12,12))
  dist_0 = dist
  for i in range(11):
      nodedist[i] = dist_0
      dist_0 = step_dist(dist_0)
  return nodedist

In [ ]:
def notecorr(major, minor, nodedist):
  corr_raw = np.zeros(24)
  for i in range(11):
    corr_mat_major = np.corrcoef(nodedist[i], major)
    corr_raw[i] = corr_mat_major[1][0]
    corr_mat_minor = np.corrcoef(nodedist[i], minor)
    corr_raw[i + 12] = corr_mat_minor[1][0]
  return corr_raw

In [ ]:
song = '/content/Indivision_feat_Boxplot_-_Interceptor_64804447.mp3'
dist = pitchdistextract(chromaextract(song))

In [ ]:
print(dist)

[ 573  105  976 1750   57 1260   81 2388  124  506 1604   89]


In [ ]:
corr_raw = notecorr(major, minor, notedistinit(dist))
print(corr_raw)
print(np.argmax(corr_raw))

[ 0.19885393 -0.22666706 -0.09184067  0.74574352 -0.68980941  0.3135902
 -0.32936744  0.29292854  0.0751663  -0.55448076  0.71734457  0.
  0.48419649 -0.65153871  0.26431818  0.16165533 -0.04160577  0.09234055
 -0.5681136   0.88209566 -0.27565787 -0.22273679  0.1228872   0.        ]
19


In [ ]:
prediction = corr_raw.argsort()[-3:][::-1]

In [ ]:
note_val = ['C major', 'C# major', 'D major', 'D# major', 'E major', 'F major', 'F# major', 'G major', 'G# major', 'A major', 'A# major', 'B major', 'C minor', 'C# minor', 'D minor', 'D# minor', 'E minor', 'F minor', 'F# minor', 'G minor', 'G# minor', 'A minor', 'A# minor', 'B minor']
print('top 3 candidates(best-to-worst):', note_val[prediction[0]], note_val[prediction[1]], note_val[prediction[2]])

top 3 candidates(best-to-worst): G minor D# major A# major


name - ground truth - predicted top 1

sunking state of mind - Bmajor - Bmajor

artificial intelligence holding out - B#minor - A#minor

Feint, Veela The journey - F#Major - F#major

Indivision, Boxplot Interceptor - Gminor - Gminor

Justice civilization - Aminor - Aminor

Pendulum Hold your colour - Fminor - Eminor

Priority one City needs sleep - F#minor - F#minor

Priority one hunted - Bminor - Aminor

rob gasser taking over - D#minor - Emajor

the prototypes kill the silence - D#major - Fminor

antonio vivaldi four seasons fall - Amajor - Fmajor

black sabbath paranoid - Eminor - Eminor

eoin o broin deep space - F#major - C#major

s.p.y. rock da house - D#major - Eminor 

kino v nashih glazah - Bmajor - Eminor

B-complex beautiful lies - Amajor - Dminor

Correct/minor error/incorrect: 6/3/7